# Tarea 03: Evaluación de funciones

Fecha *límite* de entrega: **martes 28 de febrero del 2017, antes de las 15:00 horas**.

**No** se recibirán tareas enviadas después del inicio de la clase.

Manda un solo notebook de Jupyter con tus respuestas a las preguntas al ayudante.

Explica brevemente qué estás haciendo. Redacta oraciones completas en español, usando acentos. Utiliza ecuaciones con notación LaTeX cuando sea necesario.

El notebook debe ser ejecutable, es decir, cada celda debe correr sin errores, y debe reproducir la salida que aparece en la pantalla.

## 1. Polinomios 

En esta tarea, veremos cómo evaluar ciertas funciones en la computadora.

Las funciones más fáciles de tratar son los **polinomios**.

[1] Utiliza notación LaTeX en una celda de Markdown para escribir un polinomio $p_n$ de grado $n$ con coeficientes 
$a_0$, $a_1$, $\ldots$, $a_n$, donde $a_i$ es el coeficiente de $x^i$.

### [1]

$$p(x)=a_0 + a_1 x + a_2 x^2 + \dots +a_n x^n = \sum_{i=0}^n a_i x^i$$

[2] (i) Escribe una función `evaluar_polinomio` que evalúa un polinomio de grado $2$, $p(x) = a + bx + c x^2$. La función debe aceptar como argumentos los valores de $a$, $b$ y $c$, así como el valor de $x$ donde evaluar. Verifica que funciona con el polinomio $p(x) = 1 + 2x - 3x^2$. [Pista, aquí la función se llama como sigue:

    evaluar_polinomio(a, b, c, x).
    
]

(ii) Escribe una función `evaluar_polinomio` evalúa un polinomio de grado $n$.
Acepta un arreglo `a` de $n+1$ coeficients, en el orden `a_0`, `a_1`, etc., así como un valor `x`, y evalúa $p(x)$. [Pista: Recuerda que el $i$-ésimo elemento de un arreglo se accesa con `a[i]`, y el número de entradas de un arreglo es `length(a)`.] Verifica que funciona con el polinomio $p(x) = 1 + 2x - 3x^2$. [Pista: aquí, la función se llama como sigue: 

    evaluar_polinomio([1, 2, -3], x)
]

(iii) Verifica que funciona con un polinomio cúbico.

In [1]:
function evaluar_polinomio(a,b,c,x)
    return c*x^2 + b*x + a 
end
for i in rand(20)
    if evaluar_polinomio(1,2,-3,i)!=-3*i^2 + 2*i + 1
        println("fail")
        break
    end
end
function evaluar_polinomio2(A,x)
    total=0
    for i in eachindex(A)
        total+= A[i]*x^(i-1)
    end
    return total
end
for i in rand(20)
    if evaluar_polinomio2([1,0,4,-9],i)-(-9*(i^3)+4*(i^2) + 1) >1e-10
        println("fail")
        break
    end
end

In [2]:
A=[2,4,9]
A[end-1]

4

[3] Otra manera de evaluar un polinomio es con el llamado [**algoritmo de Horner**](https://es.wikipedia.org/wiki/Algoritmo_de_Horner). La idea es que es ineficiente calcular `xˆ3` desde cero, si ya contamos con el resultado de `xˆ2`.

(i) Escribe una función para evaluar una función `evaluar_actualizar` usando esta idea: empieza desde el coeficiente de $x^0$, y guarda el valor actual de la potencia de `x` en una variable que vas actualizando.

(ii) Utiliza la descripción del algoritmo en la liga para escribir una función `evaluar_horner` que implementa el método de Horner (que se supone es aún más eficiente que la (i)), con la misma estructura que la función de la pregunta 2(ii). Verifica que da las mismas respuestas que las funciones de la pregunta 2.

In [3]:
function evaluar_actualizar(A,x)
    total=A[1]
    z=1
    for i in 2:length(A)
        z*=x
        total+=A[i]*z
    end
    return total
end
function evaluar_horner(A,x)
    total=A[end]
    for i in 1:(length(A)-1)
        total=total*x+A[end-i]
    end
    return total
end
A=[2,3,6,9,8]
for i in rand(30)
    if evaluar_horner(A,i)-evaluar_polinomio2(A,i)>1e-10
        println("fail")
        break
    end
end

[4] ¿Cuál de estos tres algoritmos es "mejor"? Aquí, por "mejor" entenderemos "más rápido".
Para verificarlo, utiliza el paquete `BenchmarkTools.jl`, y utiliza `@benchmark f($a, $x)` para ver cuánto tiempo se tarda cada función. (Debes poner explícitamente los signos `$`.) 

In [4]:
using BenchmarkTools, Plots, Interact
gr()

Plots.GRBackend()

In [5]:
@benchmark evaluar_horner($B,$0)

LoadError: UndefVarError: B not defined

In [6]:
@benchmark evaluar_actualizar($B,$0)


LoadError: UndefVarError: B not defined

In [7]:
@benchmark evaluar_polinomio2($B,$0)

LoadError: UndefVarError: B not defined

## 2. Funciones elementales

¿Cómo evalúa una computadora la función exponencial? No hay ninguna forma fácil y exacta de evaluarla. Por eso debemos usar **aproximaciones**. Normalmente, aproximamos las funciones complicadas (o sea, ¡las que no sean polinomios!) por las únicas funciones con las cuales sí sabemos trabajar, los polinomios.

Un primer tipo de aproximación es mediante las **series de Taylor**. (Ojo: no es la mejor solución, ni la que se utiliza realmente.)

[5] Recuerda que la función exponencial se define mediante una serie de Taylor. Escribe esta serie de Taylor con notación LaTeX.

### [5]

$$e^x = 1 + x + \frac{x^2}{2!} + \frac{x^3}{2!} + \dots = \sum_{n=0}^{\infty} \frac{x^n}{n!}$$

Podemos aproximar la serie de Taylor con un **polinomio de Taylor** de grado $n$, que se obtiene al truncar la serie y retener sólo los términos de grado $\le n$.

[6] Escribe una función para calcular el polinomio de Taylor de la función exponencial de orden $n$, evaluada en $x$. Utiliza una de las funciones de la preguntas 2 o 3 para evaluar el polinomio.

In [8]:
#[6]
function exponencial(x,n=10)
    return evaluar_horner([1/factorial(big(i)) for i in 0:n],x)
end
exponencial(1)

2.718281801146384479717813051146384479717813051146384479717813051146384479717817

[7] ¿Cuál valor de $n$ nos da una buena aproximación? Podemos esperar que dependerá de $x$. Para saberlo, haz lo siguiente.

(i) Dibuja la función $\exp(x)$ y las aproximaciones con distintas $n$s. Puedes usar también `@manipulate`. ¿Qué valor de $n$ parece que necesites para tener una buena aproximación de `exp(1)`? Para `exp(5)`?

(ii) Escribe una función que calcula la función exponencial que va variando $n$ *hasta que* la diferencia entre $p_{n-1}(x)$ y $p_n(x)$ sea menor que una tolerancia (la cual también es argumento de la función). ¿Concuerda con tu observación de la pregunta (i)?

In [9]:
#[7]
A=-6:.05:12
@manipulate for n in 5:20
    plot(A,[e^x for x in A],label="funcion real",xlim=(A[1] -2,A[end]+2),title="comparacion para distintos n",ylabel="e^x",xlabel="x")
    plot!(A,[exponencial(x,n) for x in A],label= "aproximacion para n= $n")
end

Interact.Options{:SelectionSlider,Int64}(Signal{Int64}(12, nactions=1),"n",12,"12",Interact.OptionDict(DataStructures.OrderedDict("5"=>5,"6"=>6,"7"=>7,"8"=>8,"9"=>9,"10"=>10,"11"=>11,"12"=>12,"13"=>13,"14"=>14…),Dict(16=>"16",11=>"11",7=>"7",9=>"9",10=>"10",19=>"19",17=>"17",8=>"8",6=>"6",5=>"5"…)),Any[],Any[],true,"horizontal")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -5 
 
 
 0 
 
 
 5 
 
 
 10 
 
 
 50000 
 
 
 100000 
 
 
 150000 
 
 
 comparacion para distintos n 
 
 
 x 
 
 
 e 
 
 
 x 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 120.14,369.674 121.33,369.674 122.52,369.674 123.71,369.674 124.9,369.674 126.089,369.674 127.279,369.674 128.469,369.674 129.659,369.674 130.849,369.674 
 132.038,369.674 133.228,369.674 134.418,369.674 135.608,369.674 136.798,369.674 137.987,369.674 139.177,369.674 140.367,369.674 141.557,369.674 142.747,369.674 
 143.937,369.674 145.126,369.674 146.316,369.674 147.506,369.674 148.696,369.674 149.886,369.674 151.075,369.674 152.265,369.674 153.455,369.674 154.645,369.674 
 155.835,369.674 157.024,369.674 158.214,369.674 159.404,369.674 160.594,369.674 161.784,369.674 162.973,369.674 164.163,369.674 165.353,369.674 166.543,369.674 
 167.733,369.674 168.922,369.674 170.112,369.674 171.302,369.674 172.492,369.674 173.682,369.674 174.871,369.674 176.061,369.674 177.251,369.674 178.441,369.674 
 179.631,369.674 180.821,369.674 182.01,369.674 183.2,369.674 184.39,369.674 185.58,369.674 186.77,369.674 187.959,369.674 189.149,369.674 190.339,369.674 
 191.529,369.674 192.719,369.674 193.908,369.674 195.098,369.674 196.288,369.674 197.478,369.674 198.668,369.674 199.857,369.674 201.047,369.674 202.237,369.674 
 203.427,369.674 204.617,369.674 205.806,369.674 206.996,369.674 208.186,369.674 209.376,369.674 210.566,369.674 211.755,369.674 212.945,369.674 214.135,369.674 
 215.325,369.674 216.515,369.674 217.705,369.674 218.894,369.674 220.084,369.674 221.274,369.674 222.464,369.674 223.654,369.674 224.843,369.674 226.033,369.674 
 227.223,369.674 228.413,369.674 229.603,369.674 230.792,369.674 231.982,369.674 233.172,369.673 234.362,369.673 235.552,369.673 236.741,369.673 237.931,369.673 
 239.121,369.673 240.311,369.673 241.501,369.673 242.69,369.673 243.88,369.673 245.07,369.673 246.26,369.673 247.45,369.673 248.639,369.673 249.829,369.673 
 251.019,369.673 252.209,369.673 253.399,369.673 254.589,369.673 255.778,369.673 256.968,369.672 258.158,369.672 259.348,369.672 260.538,369.672 261.727,369.672 
 262.917,369.672 264.107,369.672 265.297,369.672 266.487,369.672 267.676,369.672 268.866,369.671 270.056,369.671 271.246,369.671 272.436,369.671 273.625,369.671 
 274.815,369.671 276.005,369.67 277.195,369.67 278.385,369.67 279.574,369.67 280.764,369.67 281.954,369.669 283.144,369.669 284.334,369.669 285.523,369.669 
 286.713,369.668 287.903,369.668 289.093,369.668 290.283,369.667 291.473,369.667 292.662,369.667 293.852,369.666 295.042,369.666 296.232,369.665 297.422,369.665 
 298.611,369.665 299.801,369.664 300.991,369.664 302.181,369.663 303.371,369.662 304.56,369.662 305.75,369.661 306.94,369.661 308.13,369.66 309.32,369.659 
 310.509,369.658 311.699,369.658 312.889,369.657 314.079,369.656 315.269,369.655 316.458,369.654 317.648,369.653 318.838,369.652 320.028,369.651 321.218,369.649 
 322.407,369.648 323.597,369.647 324.787,369.645 325.977,369.644 327.167,369.642 328.357,369.641 329.546,369.639 330.736,369.637 331.926,369.635 333.116,369.633 
 334.306,369.631 335.495,369.629 336.685,369.627 337.875,369.624 339.065,369.622 340.255,369.619 341.444,369.616 342.634,369.613 343.824,369.61 345.014,369.607 
 346.204,369.604 347.393,369.6 348.583,369.596 349.773,369.592 350.963,369.588 352.153,369.584 353.342,369.579 354.532,369.574 355.722,369.569 356.912,369.564 
 358.102,369.558 359.291,369.552 360.481,369.546 361.671,369.539 362.861,369.532 364.051,369.525 365.241,369.517 366.43,369.509 367.62,369.501 368.81,369.492 
 370,369.483 371.19,369.473 372.379,369.462 373.569,369.452 374.759,369.44 375.949,369.428 377.139,369.416 378.328,369.402 379.518,369.388 380.708,369.374 
 381.898,369.358 383.088,369.342 384.277,369.325 385.467,369.307 386.657,369.288 387.847,369.269 389.037,369.248 390.226,369

In [10]:
A=5:25
scatter(A,[abs(e-exponencial(1,n)) for n in A],alpha=.6,yscale=:log10,label="x=1",title="aproximacion como función de n",xlabel="n",ylabel="(e^x) - exponencial (x,n)")
scatter!(A,[abs(e^5-exponencial(5,n)) for n in A],alpha=.6,label="x=5")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 20 
 
 
 10 
 
 
 - 
 
 
 20 
 
 
 10 
 
 
 - 
 
 
 10 
 
 
 10 
 
 
 0 
 
 
 aproximacion como función de n 
 
 
 n 
 
 
 e 
 
 
 x 
 
 
 - exponencial (x,n) 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 x=1 
 
 
 
 
 
 
 x=5

#### [7]

Podemos observar que , mientras que para $x=1$, nuestra función se acerca mucho al valor de $e$ desde $n \geq 5$, para $x=5$ necesitamos que $n\geq15$ para tener una buena aproximación al valor de $e^5$

In [11]:
function exponencial2(x,tol)
    n=1
    while abs(exponencial(x,n) - exponencial(x,n+1))>tol
        n+=1
    end
    return (exponencial(x,n),n)
end

exponencial2 (generic function with 1 method)

In [12]:
println(exponencial2(5,1e-5))
println(exponencial2(1,1e-5))

(1.484131470673818163608575869768520968052112543697351759628002158802653139439201e+02,20)
(2.718278769841269841269841269841269841269841269841269841269841269841269841269832,8)


### [7]

Lo observado aquí concuerda con la respuesta de la pregunta (i), los largos valores de n encontrados aquí se deben a la tolerancia requerida.

[8] Lo que acabamos de hacer es muy ineficiente -- ¿por qué?

Una mejor manera de hacerlo es parecido al método de Horner: vamos guardando el valor actual del $i$-ésimo término y lo actualizamos durante el cálculo.

(i) Implementa la función exponencial así, hasta que el tamaño del nuevo término sea menor que cierta tolerancia. Verifica que funciona.

In [22]:
function exponencial3(x,tol)
    total=1
    n=1
    nuevo=x
    while abs(nuevo)>tol
        total+=nuevo
        n+=1
        nuevo*=x/n
    end
    return total
end

exponencial3 (generic function with 1 method)

 in module Main at In[13]:2 overwritten at In[22]:2.


[9] La función exponencial también sirve para números complejos. En Julia, $i$, la raíz cuadrada de $-1$, se escribe como `im`.

(i) Utiliza esto para escribir una función `mi_sin(x)` para calcular $\sin(x)$.

In [38]:
#[9]
function mi_sin(x,tol)
    return ((exponencial3(im*x,tol)-exponencial3(-im*x,tol))/(2*im))
end
mi_sin(π/2,1e-5)

1.0000035425842861 - 0.0im

Any, Any) in module Main at In[26]:3 overwritten at In[38]:3.


[10] Aunque estamos llevando a cabo cálculos con números de punto flotante, podemos lograr ciertas **garantías** sobre los resultados.

(i) Escribe el teorema de Taylor con la forma del término complementario de Lagrange para un polinomio de Taylor $p_2(x)$ de grado 2, con término complementario de orden 3.

(ii) Considera la función $\exp$ en el rango $I = [-\frac{1}{2}, \frac{1}{2}]$. Encuentra una cota superior, $d$, para el término complementario en $I$. [Pista: Cuál es el valor máximo del término complementario sobre todo el intervalo?] 

(iii) Dibuja un "tubo" entre $p_2(x) - d$ y $p_2(x) + d$. Esto representa una región en la cual, de forma **garantizada**, cae $\exp$ dentro del intervalo $I$. Dibuja $\exp$ encima.

[En `Plots.jl`, para rellenar la región entre la función `f` y la función `g`, puedes utilizar `plot(xx, f.(xx), fillrange=g.(xx), alpha=0.3)`, donde `xx` son las coordenadas `xx` de los puntos. Aquí, `alpha` corresponde al grado de transparencia de la región rellenada.]

** Opcional: ¿Qué ocurre si haces lo mismo para un grado de polinomio de Taylor superior con su cota correspondiente. Puedes utilizar `@manipulate`.

### [10]

$$f(x) =  \sum_{i=0}^n \frac{d^{(i)}f}{dx^{(i)}}\Big|_{x_0} \frac{(x-x_0)^i}{i!} +R_n =\sum_{i=0}^n \frac{d^{(i)}f}{dx^{(i)}}\Big|_{x_0} \frac{(x-x_0)^i}{i!} + \frac{d^{(n+1)}f}{dx^{(n+1)}}\Big|_{x*} \frac{(x-x_0)^{(n+1)}}{(n+1)!}$$

Para algún $x* \in (x_0,x)$. Si fijamos la serie en $n=2$, obtenemos de manera explícita la siguiente fórmula:

$$f(x)= f(x_0) + \frac{d^{(1)}f}{dx^{(1)}}\Big|_{x_0} (x-x_0) +\frac{d^{(2)}f}{dx^{(2)}}\Big|_{x_0} \frac{(x-x_0)^2}{2!} + \frac{d^{(3)}f}{dx^{(3)}}\Big|_{x*} \frac{(x-x_0)^{3}}{3!}$$

Utilizando esta fórmula, podemos encontrar un valor máximo para el término complementario para la función $e^x$, si fijamos $x_0=0$, como:

$$R_n=e^x\Big|_{x*} \frac{x^{3}}{3!}$$

Al ser $e^x$ una función estrictamente creciente, entonces, para todos los x en el intervalo $[-\frac{1}{2},\frac{1}{2}]$, el término residual alcanzará un valor máximo cuando $x,x*=\frac{1}{2}$, cuyo valor es de .0343483

In [40]:
d=e^(1/2)*((1/2)^3)/6

0.03434835980625267

In [46]:
A=-1/2:.005:1/2
plot(A,[exponencial(x,2)+d for x in A],fillrange=[exponencial(x,2)-d for x in A],alpha=.4,label="rango de error",xlabel="x",ylabel="e^x",title="Aproximación")
plot!(A,[e^x for x in A], label="funcion real")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -0.50 
 
 
 -0.25 
 
 
 0.00 
 
 
 0.25 
 
 
 0.50 
 
 
 1.0 
 
 
 1.5 
 
 
 Aproximación 
 
 
 x 
 
 
 e 
 
 
 x 
 
<polygon clip-path="url(#clip02)" points="
45.8815,347.414 48.6324,346.6 51.3833,345.778 54.1342,344.947 56.8851,344.109 59.636,343.262 62.3869,342.408 65.1378,341.545 67.8887,340.674 70.6396,339.795 
 73.3905,338.908 76.1414,338.013 78.8923,337.11 81.6433,336.199 84.3942,335.279 87.1451,334.352 89.896,333.416 92.6469,332.472 95.3978,331.52 98.1487,330.56 
 100.9,329.592 103.651,328.616 106.401,327.632 109.152,326.64 111.903,325.639 114.654,324.631 117.405,323.614 120.156,322.589 122.907,321.556 125.658,320.515 
 128.409,319.466 131.16,318.409 133.91,317.344 136.661,316.271 139.412,315.189 142.163,314.1 144.914,313.002 147.665,311.896 150.416,310.782 153.167,309.66 
 155.918,308.53 158.669,307.392 161.42,306.246 164.17,305.091 166.921,303.929 169.672,302.758 172.423,301.58 175.174,300.393 177.925,299.198 180.676,297.995 
 183.427,296.784 186.178,295.565 188.929,294.338 191.68,293.102 194.43,291.859 197.181,290.607 199.932,289.347 202.683,288.08 205.434,286.804 208.185,285.52 
 210.936,284.228 213.687,282.928 216.438,281.619 219.189,280.303 221.94,278.978 224.69,277.646 227.441,276.305 230.192,274.956 232.943,273.599 235.694,272.234 
 238.445,270.861 241.196,269.48 243.947,268.091 246.698,266.694 249.449,265.288 252.2,263.874 254.95,262.453 257.701,261.023 260.452,259.585 263.203,258.139 
 265.954,256.685 268.705,255.223 271.456,253.752 274.207,252.274 276.958,250.788 279.709,249.293 282.46,247.79 285.21,246.279 287.961,244.761 290.712,243.234 
 293.463,241.698 296.214,240.155 298.965,238.604 301.716,237.045 304.467,235.477 307.218,233.901 309.969,232.318 312.719,230.726 315.47,229.126 318.221,227.518 
 320.972,225.902 323.723,224.278 326.474,222.645 329.225,221.005 331.976,219.356 334.727,217.7 337.478,216.035 340.229,214.362 342.979,212.681 345.73,210.992 
 348.481,209.295 351.232,207.59 353.983,205.877 356.734,204.155 359.485,202.426 362.236,200.688 364.987,198.942 367.738,197.189 370.489,195.427 373.239,193.657 
 375.99,191.878 378.741,190.092 381.492,188.298 384.243,186.495 386.994,184.685 389.745,182.866 392.496,181.04 395.247,179.205 397.998,177.362 400.749,175.511 
 403.499,173.652 406.25,171.784 409.001,169.909 411.752,168.026 414.503,166.134 417.254,164.234 420.005,162.327 422.756,160.411 425.507,158.487 428.258,156.555 
 431.009,154.615 433.759,152.666 436.51,150.71 439.261,148.746 442.012,146.773 444.763,144.792 447.514,142.804 450.265,140.807 453.016,138.802 455.767,136.789 
 458.518,134.768 461.269,132.738 464.019,130.701 466.77,128.656 469.521,126.602 472.272,124.54 475.023,122.471 477.774,120.393 480.525,118.307 483.276,116.213 
 486.027,114.111 488.778,112 491.528,109.882 494.279,107.756 497.03,105.621 499.781,103.478 502.532,101.328 505.283,99.1686 508.034,97.0017 510.785,94.8266 
 513.536,92.6434 516.287,90.4522 519.038,88.2528 521.788,86.0453 524.539,83.8297 527.29,81.6061 530.041,79.3743 532.792,77.1344 535.543,74.8864 538.294,72.6304 
 541.045,70.3662 543.796,68.0939 546.547,65.8135 549.298,63.525 552.048,61.2285 554.799,58.9238 557.55,56.611 560.301,54.2901 563.052,51.9611 565.803,49.624 
 568.554,47.2789 571.305,44.9256 574.056,42.5642 576.807,40.1947 579.558,37.8171 582.308,35.4314 585.059,33.0376 587.81,30.6357 590.561,28.2257 593.312,25.8076 
 596.063,23.3815 596.063,45.6414 593.312,48.0676 590.561,50.4857 587.81,52.8957 585.059,55.2976 582.308,57.6914 579.558,60.0771 576.807,62.4547 574.056,64.8242 
 571.305,67.1856 568.554,69.5388 565.803,71.884 563.052,74.2211 560.301,76.5501 557.55,78.871 554.799,81.1838 552.048,83.4884 549.298,85.785 546.547,88.0735 
 543.796,90.3539 541.045,92.6262 538.294,94.8903 535.543,97.1464 532.792,99.3944 530.041,101.634 527.29,103.866 524.539,106.09 521.788,108.305 519.038,110.513 
 516.287,112.712 513.536,114.903 510.785,117.087 508.034,119.262 50

In [48]:
@manipulate for n in 2:10
    d=e^(1/2)*((1/2)^n)/factorial(big(n))
    plot(A,[exponencial(x,n)+d for x in A],fillrange=[exponencial(x,n)-d for x in A],alpha=.4,label="rango de error, n=$n",xlabel="x",ylabel="e^x",title="Aproximación")
    plot!(A,[e^x for x in A], label="funcion real")
end

Interact.Options{:SelectionSlider,Int64}(Signal{Int64}(6, nactions=1),"n",6,"6",Interact.OptionDict(DataStructures.OrderedDict("2"=>2,"3"=>3,"4"=>4,"5"=>5,"6"=>6,"7"=>7,"8"=>8,"9"=>9,"10"=>10),Dict(7=>"7",4=>"4",9=>"9",10=>"10",2=>"2",3=>"3",5=>"5",8=>"8",6=>"6")),Any[],Any[],true,"horizontal")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -0.50 
 
 
 -0.25 
 
 
 0.00 
 
 
 0.25 
 
 
 0.50 
 
 
 1.0 
 
 
 1.5 
 
 
 Aproximación 
 
 
 x 
 
 
 e 
 
 
 x 
 
<polygon clip-path="url(#clip02)" points="
45.8815,369.65 48.6324,368.64 51.3833,367.625 54.1342,366.605 56.8851,365.579 59.636,364.549 62.3869,363.513 65.1378,362.472 67.8887,361.426 70.6396,360.375 
 73.3905,359.318 76.1414,358.256 78.8923,357.189 81.6433,356.117 84.3942,355.039 87.1451,353.955 89.896,352.867 92.6469,351.772 95.3978,350.673 98.1487,349.567 
 100.9,348.457 103.651,347.34 106.401,346.218 109.152,345.091 111.903,343.957 114.654,342.819 117.405,341.674 120.156,340.524 122.907,339.367 125.658,338.206 
 128.409,337.038 131.16,335.864 133.91,334.685 136.661,333.499 139.412,332.308 142.163,331.111 144.914,329.907 147.665,328.698 150.416,327.483 153.167,326.261 
 155.918,325.033 158.669,323.8 161.42,322.56 164.17,321.314 166.921,320.061 169.672,318.802 172.423,317.537 175.174,316.266 177.925,314.988 180.676,313.704 
 183.427,312.414 186.178,311.117 188.929,309.813 191.68,308.503 194.43,307.186 197.181,305.863 199.932,304.533 202.683,303.197 205.434,301.854 208.185,300.504 
 210.936,299.147 213.687,297.783 216.438,296.413 219.189,295.036 221.94,293.652 224.69,292.261 227.441,290.863 230.192,289.457 232.943,288.045 235.694,286.626 
 238.445,285.2 241.196,283.766 243.947,282.326 246.698,280.878 249.449,279.423 252.2,277.96 254.95,276.491 257.701,275.014 260.452,273.529 263.203,272.037 
 265.954,270.538 268.705,269.031 271.456,267.516 274.207,265.994 276.958,264.465 279.709,262.927 282.46,261.382 285.21,259.829 287.961,258.269 290.712,256.7 
 293.463,255.124 296.214,253.54 298.965,251.948 301.716,250.348 304.467,248.739 307.218,247.123 309.969,245.499 312.719,243.866 315.47,242.226 318.221,240.577 
 320.972,238.92 323.723,237.254 326.474,235.581 329.225,233.898 331.976,232.208 334.727,230.509 337.478,228.801 340.229,227.085 342.979,225.36 345.73,223.627 
 348.481,221.885 351.232,220.134 353.983,218.375 356.734,216.606 359.485,214.829 362.236,213.043 364.987,211.248 367.738,209.443 370.489,207.63 373.239,205.808 
 375.99,203.977 378.741,202.136 381.492,200.286 384.243,198.427 386.994,196.559 389.745,194.681 392.496,192.794 395.247,190.897 397.998,188.991 400.749,187.075 
 403.499,185.15 406.25,183.215 409.001,181.271 411.752,179.316 414.503,177.352 417.254,175.378 420.005,173.394 422.756,171.4 425.507,169.396 428.258,167.382 
 431.009,165.358 433.759,163.324 436.51,161.28 439.261,159.225 442.012,157.16 444.763,155.085 447.514,152.999 450.265,150.903 453.016,148.797 455.767,146.68 
 458.518,144.552 461.269,142.413 464.019,140.264 466.77,138.104 469.521,135.934 472.272,133.752 475.023,131.559 477.774,129.356 480.525,127.141 483.276,124.915 
 486.027,122.679 488.778,120.43 491.528,118.171 494.279,115.9 497.03,113.618 499.781,111.325 502.532,109.02 505.283,106.703 508.034,104.375 510.785,102.035 
 513.536,99.6839 516.287,97.3205 519.038,94.9453 521.788,92.5582 524.539,90.1592 527.29,87.7481 530.041,85.3249 532.792,82.8896 535.543,80.4421 538.294,77.9823 
 541.045,75.5102 543.796,73.0257 546.547,70.5288 549.298,68.0193 552.048,65.4972 554.799,62.9625 557.55,60.4151 560.301,57.855 563.052,55.282 565.803,52.6961 
 568.554,50.0972 571.305,47.4854 574.056,44.8604 576.807,42.2223 579.558,39.5709 582.308,36.9063 585.059,34.2283 587.81,31.5369 590.561,28.832 593.312,26.1135 
 596.063,23.3815 596.063,23.4052 593.312,26.1373 590.561,28.8558 587.81,31.5607 585.059,34.2521 582.308,36.9301 579.558,39.5947 576.807,42.246 574.056,44.8842 
 571.305,47.5091 568.554,50.121 565.803,52.7199 563.052,55.3058 560.301,57.8788 557.55,60.4389 554.799,62.9863 552.048,65.521 549.298,68.0431 546.547,70.5525 
 543.796,73.0495 541.045,75.534 538.294,78.0061 535.543,80.4659 532.792,82.9134 530.041,85.3487 527.29,87.7719 524.539,90.183 521.788,92.582 519.038,94.9691 
 516.287,97.3443 513.536,99.7076 510.785,102.059 508.034,104.399 505.